Carga el conjunto de datos MNIST y divídelo en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, utiliza 50,000 instancias para el entrenamiento, 10,000 para la validación y 10,000 para las pruebas). 

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [2]:
from sklearn.datasets import fetch_openml

X_mnist, y_mnist = fetch_openml('mnist_784', return_X_y=True, as_frame=False, parser='auto')

Entrena clasificadores individuales, por ejemplo un clasificador RandomForest, un clasificador SVM y un MLP. 

In [3]:
#escalo los datos
X_minist = X_mnist / 255.0

#separo datos de entrenamiento (50) y test (10) y validacion (10)
X_train, X_test, X_val = X_mnist[:50000], X_mnist[50000:60000], X_mnist[60000:]
y_train, y_test, y_val = y_mnist[:50000], y_mnist[50000:60000], y_mnist[60000:]


In [4]:
# Random Forest
#Creo el modelo
rnd_clf = RandomForestClassifier(n_estimators = 500, #500 arboles
                                 bootstrap = True, #con reemplazo
                                 max_leaf_nodes = 16,#maximo 16 nodos hojas
                                 n_jobs = -1, 
                                 max_features = 'sqrt',
                                 oob_score = True,#para que devuelva el desempeño out of bag
                                 random_state = 0)
rnd_clf.fit(X_train, y_train)

#analizo el score del modelo con los datos de test
rnd_clf.oob_score_

0.81824

In [5]:
#clasificador svm:
from sklearn.svm import SVC

#creo el modelo
svm_clf = SVC(C = 0.2, gamma= 0.6, random_state= 42, tol= 2)
svm_clf.fit(X_train, y_train)

#veo el score del modelo con los datos de test
svm_clf.score(X_test, y_test)

0.1064

In [6]:
#clasificador mlp(perceptron multicapa):
from sklearn.neural_network import MLPClassifier

#creo el modelo
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), random_state=42)
mlp_clf.fit(X_train, y_train)

#veo el score del modelo con los datos de test
mlp_clf.score(X_test, y_test)

0.9655

A continuación, intenta combinarlos en un ensamble cuyo desempeño supere a cada clasificador individual en el conjunto de validación, utilizando votación por mayoría hard y soft. Una vez que hayas encontrado uno, pruébalo en el conjunto de prueba.

In [7]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

voting_clf = VotingClassifier(
    estimators=[
        ('svm', SVC(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('mlp', MLPClassifier(random_state=42))
    ]
)

voting_clf.fit(X_train, y_train)

voting_clf.score(X_test, y_test)

0.9794

In [8]:
#soft voting: 

voting_clf.voting = "soft"
voting_clf.named_estimators["svm"].probability = True #cambio para que pueda usar las probabilidades
voting_clf.fit(X_train, y_train)
voting_clf.score(X_test, y_test)

0.9774

Prueba otros ensambles usando Bagging, Boosting y Stacking (Stacking utiliza validación cruzada por lo que es mejor dividir los datos sólo en entrenamiento y prueba)

In [9]:
#Bagging:
#la diversidad del ensamble se obtiene mediante subconjuntos muestreados de forma aleatoria del conjunto de datos original con reemplazo
#(pueden repetirse los datos). Estos subconjuntos, se utilizan para entrenar diferentes clasificadores del mismo tipo. 
#La predicción final, se obtiene a partir de las predicciones individuales utilizando en general votación por mayoría.


from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(estimator= DecisionTreeClassifier(), 
                            n_estimators=500, #500 arboles
                            max_samples=100, #100 n max de muestras de los datos de entrenamiento
                            max_features = 1.0, #1.0 porcentaje de caracteristicas-->bagging
                            bootstrap = True, #con reemplazo
                            n_jobs=-1, #no limito el crecimiento
                            random_state=0) #semilla
bag_clf.fit(X_train, y_train) #se entrenan de forma independiente


print(bag_clf.score(X_test, y_test))

0.8644


In [ ]:
#Boosting:
#la forma en como un ensamble puede potenciar (boost) un clasificador 'débil' para obtener uno más robusto.
#los clasificadores se entrenan en serie (consecutivamente) y la performance del clasificador k influencia en el entrenamiento
#La idea es entrenar clasificadores secuencialmente y que cada clasificador corrija a su clasificador predecesor.

from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier( SVC(C=0.2, gamma=0.6, probability=True, random_state=42, tol = 2), 
                              n_estimators=100,
                              learning_rate=1, 
                              random_state=42,
                              algorithm='SAMME.R'
                             )
ada_clf.fit(X_train, y_train)
#SAMME _(Stagewise Additive Modeling using a Multiclass Exponential loss function)_ Si los clasificadores en el ensamble
# pueden realizar predicciones mediante probabilidades,
# sklearn utiliza la variante SAMME.R que tiene en cuenta las probabilidades en lugar de las predicciones
# generalmente obtiene mejores desempeños.

ada_clf.score(X_test, y_test)

In [10]:
#stacking:
#combinar clasificadores con el fin de reducir sus sesgos. Las predicciones de cada clasificador individual se apilan
#se utilizan como entrada para entrenar un clasificador final que calcula la predicción


X_train_2, X_test_2 = X_mnist[:60000], X_mnist[60000:]
y_train_2, y_test_2 = y_mnist[:60000], y_mnist[60000:]

from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ('svm', SVC(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('mlp',MLPClassifier(random_state=42))
    ],
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5
)
stacking_clf.fit(X_train, y_train) #se entrena por validacion cruzada

stacking_clf.score(X_test, y_test)

KeyboardInterrupt: 